# Imports

In [1]:
import numpy as np
from datetime import datetime 
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torch import autograd

import torchvision
from torchvision import datasets, transforms

import matplotlib.pyplot as plt

from sklearn.svm import SVC # For SVM

%load_ext tensorboard

# check if CUDA is available
is_gpu_available = torch.cuda.is_available()

if not is_gpu_available:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

DIR_PATH = '/content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526'

CUDA is available!  Training on GPU ...


# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

drive_path = '/content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526'

# Path for check point
ckpts_path = os.path.join(drive_path, 'Q3_GAN', 'checkpoints')
os.makedirs(ckpts_path, exist_ok=True)

print('Drive Path : ' + drive_path)
print('Check points Path : ' + ckpts_path)

# Define models Consts

In [1]:
DIM = 64
BATCH_SIZE = 64
NOISE_SIZE = 128
GEN_ITER = 20000
CRITIC_ITER = 5
OUTPUT_SIZE = 784  # Fashion MNIST size (28, 28)
IMAGE_CHANNELS = 1
LAMBDA = 10 # Gradient penalty hyperparameter


# Define GAN Model

In [ ]:
from torch.nn.modules.batchnorm import BatchNorm2d
class Generator(nn.Module):
  def __init__(self, mode, kernel_size_1=5, kernel_size_2=5):
    super(Generator, self).__init__()
    self.mode = mode
    self.fc1 = nn.Sequential(
        nn.Linear(NOISE_SIZE, 4 * 4 * 4 * DIM),
        nn.BatchNorm2d(4 * 4 * 4 * DIM) if mode == 'wgan' else nn.Identity(),
        nn.ReLU(True)
        )
    
    self.pre_activate1 = nn.Sequential(
        nn.ConvTranspose2d(4 * DIM, 2 * DIM, kernel_size_1),
        nn.BatchNorm2d(2 * DIM),
        nn.ReLU(True)
    )

    self.pre_activate2 = nn.Sequential(
        nn.ConvTranspose2d(2 * DIM, DIM, kernel_size_2),
        nn.BatchNorm2d(2 * DIM),
        nn.ReLU(True)
    )

    self.pre_activate3 = nn.ConvTranspose2d(DIM, IMAGE_CHANNELS, stride=2)

    self.sigmoid = nn.Sigmoid

  def forward(self, input):
    output = self.fc1(input)
    output = output.view(-1, 4 * DIM, 4, 4)
    output = self.pre_activate1(output)
    print(output.size())
    output = output[:, :, :7, :7]
    print(output.size())
    output = self.pre_activate2(output)
    print(output.size())
    output = self.pre_activate3(output)
    print(output.size())
    output = self.sigmoid(output)
    return output.view(-1, OUTPUT_SIZE)
  
class Discriminator(nn.Module):
  def __init__(self, kernel_size=5):
    super(Discriminator, self).__init__()

    self.fc1 = nn.Sequential(
        nn.Linear(IMAGE_CHANNELS, DIM, kernel_size, stride=2, padding=2),
        nn.ReLU(True)
    )

    self.fc2 = nn.Sequential(
        nn.Linear(DIM, 2 * DIM, kernel_size, stride=2, padding=2),
        nn.ReLU(True)
    )

    self.fc3 = nn.Sequential(
        nn.Linear(2 * DIM, 4 * DIM, kernel_size, stride=2, padding=2),
        nn.ReLU(True)
    )

    self.fc4 = nn.Linear(4 * 4 * 4 * DIM, 1)

  def forward(self, input):
    output = self.fc1(input)
    output = self.fc2(output)
    output = self.fc3(output)
    output = output.view(-1, 4 * 4 * 4 * DIM)
    output = self.fc4(output)
    return output.view(-1)


def calc_gradient_penalty(disc, real_data, fake_data):
  alpha = torch.rand(BATCH_SIZE, 1)
  alpha = alpha.expand(real_data.size())
  alpha = alpha.to(DEVICE) if torch.cuda.is_available() else alpha

  interpolates = alpha * real_data + ((1 - alpha) * fake_data)

  if torch.cuda.is_available():
      interpolates = interpolates.to(DEVICE)
  interpolates = autograd.Variable(interpolates, requires_grad=True)

  disc_interpolates = disc(interpolates)

  gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                            grad_outputs=torch.ones(disc_interpolates.size()).to(DEVICE) if torch.cuda.is_available() else torch.ones(
                                disc_interpolates.size()),
                            create_graph=True, retain_graph=True, only_inputs=True)[0]

  gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
  return gradient_penalty
